
# A gentle introduction to type classes in Scala

###  Functors, monads and other functional programming patterns as a way to structure program design

In this talk we will cover
* what type classes are
* where they are come from
* what are the common patterns
* how they are supported in Scala

In computer science, a type class is a type system construct that supports ad hoc polymorphism. 

This is achieved by adding constraints to type variables in parametrically polymorphic types. Such a constraint typically involves a type class `T` and a type variable `a`, and means that `a` can only be instantiated to a type whose members support the overloaded operations associated with `T`. 

https://en.wikipedia.org/wiki/Type_class

<center>
<img src="img/scala-book.jpg" height="600">
</center>

<center>
<img src="img/scala-fp.png" height="800">
</center>

<center>
<img src="img/haskell-book.jpg" height="800">
</center>

<center>
<img src="img/bartosz.jpg" height="800">
</center>

In [ ]:
class Person(val name: String, val age: Int)

In [ ]:
val ann = new Person("Ann", 30)

In [ ]:
println(ann)

In [ ]:
class Person(val name: String, val age: Int) {
    override def toString: String = s"Person(name: $name, age: $age)"
}

In [ ]:
val ann = new Person("Ann", 30)

In [ ]:
val anotherAnn = new Person("Ann", 30)
val bob = new Person("Bob", 40)

In [ ]:
println(ann == anotherAnn)

println(ann == bob)

In [ ]:
class Person(val name: String, val age: Int) {
    override def toString: String = s"Person(name: $name, age: $age)"
    
    override def hashCode: Int = 41 * name.hashCode + age.hashCode
    
    override def equals(other: Any): Boolean = other match {
        case that: Person => name == that.name && age == that.age
        case _ => false
    }
}

In [ ]:
val ann = new Person("Ann", 30)
val anotherAnn = new Person("Ann", 30)
val bob = new Person("Bob", 40)

In [ ]:
println(ann == anotherAnn)

println(ann == bob)

In [ ]:
ann <= anotherAnn

For ordering we can mix in the Ordered trait.

By defining a single `compare` method, the trait automatically provides implementation for all the comparison operators `<`, `<=`, `>`, `>=`.

In [ ]:
class Person(val name: String, val age: Int) extends Ordered[Person] {
    override def toString: String = s"Person(name: $name, age: $age)"
    
    override def hashCode: Int = 41 * name.hashCode + age.hashCode
    
    override def equals(other: Any): Boolean = other match {
        case that: Person => name == that.name && age == that.age
        case _ => false
    }
    
    def compare(other: Person): Int = this.age - other.age
}

In [ ]:
val ann = new Person("Ann", 30)
val anotherAnn = new Person("Ann", 30)
val bob = new Person("Bob", 40)
val joe = new Person("Joe", 35)

In [ ]:
println(ann < bob)
println(bob < joe)

In [ ]:
def insertionSortUpperBound[T <: Ordered[T]](xs: List[T]): List[T] = {
    def insert(item: T, ys: List[T]): List[T] = ys match {
        case Nil => List(item)
        case y :: yss => if (item < y) item :: ys else y :: insert(item, yss) 
    }
    
    xs match {
        case Nil => Nil
        case x :: xss => insert(x, insertionSortUpperBound(xss))
    }
}

In [ ]:
insertionSortUpperBound(List(bob, ann, joe, anotherAnn))

In [ ]:
insertionSortUpperBound(List(4, 3, 5, 1))

In [ ]:
class Person(val name: String, val age: Int) {
    override def toString: String = s"Person(name: $name, age: $age)"
    
    override def hashCode: Int = 41 * name.hashCode + age.hashCode
    
    override def equals(other: Any): Boolean = other match {
        case that: Person => name == that.name && age == that.age
        case _ => false
    }
}

object Person {
    implicit val ageOrdering: Ordering[Person] = new Ordering[Person] {
        def compare(p1: Person, p2: Person): Int = p1.age compare p2.age
    }
}

In [ ]:
val ann = new Person("Ann", 30)
val anotherAnn = new Person("Ann", 30)
val bob = new Person("Bob", 40)
val joe = new Person("Joe", 35)

In [ ]:
def insertionSortOrdering[T](xs: List[T])(implicit ord: Ordering[T]): List[T] = {
    def insert(item: T, ys: List[T]): List[T] = ys match {
        case Nil => List(item)
        case y :: yss => 
            if (ord.lt(item, y)) item :: ys 
            else y :: insert(item, yss) 
    }
    
    xs match {
        case Nil => Nil
        case x :: xss => insert(x, insertionSortOrdering(xss))
    }
}

In [ ]:
insertionSortOrdering(List(bob, ann, joe, anotherAnn))

In [ ]:
insertionSortOrdering(List(bob, ann, joe, anotherAnn))(Ordering[Person].reverse)

In [ ]:
insertionSortOrdering(List(4, 3, 5, 1))

In [ ]:
def insertionSort[T: Ordering](xs: List[T]): List[T] = {
    def insert(item: T, ys: List[T]): List[T] = ys match {
        case Nil => List(item)
        case y :: yss => 
            if (implicitly[Ordering[T]].lt(item, y)) item :: ys 
            else y :: insert(item, yss)
    }
    
    xs match {
        case Nil => Nil
        case x :: xss => insert(x, insertionSort(xss))
    }
}

In [ ]:
insertionSort(List(bob, ann, joe, anotherAnn))

In [ ]:
insertionSort(List(bob, ann, joe, anotherAnn))(Ordering[Person].reverse)

In [ ]:
insertionSort(List(1L, 3L, 2L))

In [ ]:
class Person(val name: String, val age: Int) {
    override def toString: String = s"Person(name: $name, age: $age)"
    
    override def hashCode: Int = 41 * name.hashCode + age.hashCode
    
    override def equals(other: Any): Boolean = other match {
        case that: Person => name == that.name && age == that.age
        case _ => false
    }
}

object Person {
    implicit val ageOrdering: Ordering[Person] = new Ordering[Person] {
        def compare(p1: Person, p2: Person): Int = p1.age compare p2.age
    }
    
    implicit val nameOrdering: Ordering[Person] = new Ordering[Person] {
        def compare(p1: Person, p2: Person): Int = p1.name compare p2.name
    }
}

In [ ]:
val ann = new Person("Ann", 30)
val anotherAnn = new Person("Ann", 30)
val bob = new Person("Bob", 40)
val joe = new Person("Joe", 35)

In [ ]:
insertionSort(List(bob, ann, joe, anotherAnn))

In [ ]:
insertionSort(List(bob, ann, joe, anotherAnn))(Person.nameOrdering)